[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OlivierGeorgeon/Developmental-AI-Lab/blob/master/docs/agent3.ipynb)

# THE AGENT WHO TAMED THE TURTLE

# Learning objectives

Upon completing this lab, you will be able to assign appropriate valences to interactions, enabling a developmental agent to exhibit exploratory behavior in a simulated environment.

# Setup
## Import the turtle environment

In [2]:
!pip3 install ColabTurtle
from ColabTurtle.Turtle import *

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ColabTurtle: filename=colabturtle-2.1.0-py3-none-any.whl size=7675 sha256=ee163d0da9b4933f548d9e5499e3f3afa939b7c34e71783cf09cfee965649d84
  Stored in directory: /home/amine/.cache/pip/wheels/8d/30/a2/2b8e815bd3673383f33c0dd80d33ef7651a7c257c528515158
Successfully built ColabTurtle

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## Define the Agent class

In [3]:
class Agent:
    def __init__(self, _valences):
        """ Creating our agent """
        self._valences = _valences
        self._action = None
        self._predicted_outcome = None

    def action(self, _outcome):
        """ tracing the previous cycle """
        if self._action is not None:
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {_outcome}, " 
                  f"Prediction: {self._predicted_outcome == _outcome}, Valence: {self._valences[self._action][_outcome]}")

        """ Computing the next action to enact """
        # TODO: Implement the agent's decision mechanism
        self._action = 0
        # TODO: Implement the agent's anticipation mechanism
        self._predicted_outcome = 0
        return self._action


## Define the tutle environment class

You don't need to worry about the code of the ColabTurtleEnvironment below.

Just know that this environment:
* interprets the agent's actions as follows  `0`: move forward, `1`: turn left, `2`: turn right.
* returns outcome `1` when the turtle bumps into the border of the window, and `0` otherwise.

In [4]:
# @title Initialize the turtle environment

BORDER_WIDTH = 20

class ColabTurtleEnvironment:

    def __init__(self):
        """ Creating the Turtle window """
        bgcolor("lightGray")
        penup()
        goto(window_width() / 2, window_height()/2)
        face(0)
        pendown()
        color("green")

    def outcome(self, action):
        """ Enacting an action and returning the outcome """
        _outcome = 0
        for i in range(10):
            # _outcome = 0
            if action == 0:
                # move forward
                forward(10)
            elif action == 1:
                # rotate left
                left(4)
                forward(2)
            elif action == 2:
                # rotate right
                right(4)
                forward(2)

            # Bump on screen edge and return outcome 1
            if xcor() < BORDER_WIDTH:
                goto(BORDER_WIDTH, ycor())
                _outcome = 1
            if xcor() > window_width() - BORDER_WIDTH:
                goto(window_width() - BORDER_WIDTH, ycor())
                _outcome = 1
            if ycor() < BORDER_WIDTH:
                goto(xcor(), BORDER_WIDTH)
                _outcome = 1
            if ycor() > window_height() - BORDER_WIDTH:
                goto(xcor(), window_height() -BORDER_WIDTH)
                _outcome = 1

            # Change color
            if _outcome == 0:
                color("green")
            else:
                # Finit l'interaction
                color("red")
                # if action == 0:
                #     break
                if action == 1:
                    for j in range(10):
                        left(4)
                elif action == 2:
                    for j in range(10):
                        right(4)
                break

        return _outcome

## Define the valence of interactions

In [5]:
valences = [[1, -1], 
            [-1, -1],
            [-1, -1]]

The valence table specifies the valence of each interaction. An interaction is a tuple (action, outcome):

|| 0 Not bump | 1 Bump|
|---|---|---|
| 0 Forward| 1 | -1 |
| 1 Left | -1 | -1 |
| 2 Right| -1 | -1 |

## Instantiate the agent

In [6]:
a = Agent(valences)

## Run the simulation 

In [7]:
# @title Run the simulation

initializeTurtle()

# Parameterize the rendering
bgcolor("lightGray")
penup()
goto(window_width() / 2, window_height()/2)
face(0)
pendown()
color("green")
speed(10)

e = ColabTurtleEnvironment()

outcome = 0
for i in range(10):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1


Observe the turtle moving in a straigt line until it bumps into the border of the window

# PRELIMINARY EXERCISE

Copy Agent2 that you designed in your previous assignment to this notebook. 

Observe how your Agent2 behaves in this environment 

# ASSIGNMENT

Implement Agent3 by modifying your previous Agent2 such that it can select 3 possible actions: `0`, `1`, or `2`.

Choose the valences of interactions so that the agent does not remain stuck in a corner of the environment. 

## Create Agent3 by overriding the class Agent or your previous class Agent2

In [8]:
class Agent3(Agent):
    def __init__(self, _valences):
        """Creating our turtle-taming agent"""
        super().__init__(_valences)
        # Memory: stores the last observed outcome for each action
        self.memory = {}
        # Counter for consecutive correct predictions
        self.correct_count = 0
        # Boredom threshold
        self.boredom_threshold = 4
        
    def action(self, _outcome):
        """Tracing the previous cycle"""
        if self._action is not None:
            # Update memory with the observed outcome
            self.memory[self._action] = _outcome
            
            # Check if prediction was correct
            satisfied = (self._predicted_outcome == _outcome)
            
            # Update correct prediction counter
            if satisfied:
                self.correct_count += 1
            else:
                self.correct_count = 0
            
            # Calculate valence
            valence = self._valences[self._action][_outcome]
            
            # Check for boredom
            bored = (self.correct_count >= self.boredom_threshold)
            
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, "
                  f"Outcome: {_outcome}, Prediction: {satisfied}, Valence: {valence}, Bored: {bored}")
        
        """Computing the next action to enact"""
        # TODO: Implement the agent's decision mechanism
        # ✅ IMPLÉMENTATION DU MÉCANISME DE DÉCISION AVEC 3 ACTIONS
        if self.correct_count >= self.boredom_threshold:
            # Bored: try a different action
            import random
            possible_actions = [0, 1, 2]
            if self._action is not None and self._action in possible_actions:
                possible_actions.remove(self._action)
            self._action = random.choice(possible_actions)
            self.correct_count = 0
        else:
            # Not bored: choose the action with the best anticipated valence
            best_action = None
            best_valence = -float('inf')
            
            for action in [0, 1, 2]:
                if action in self.memory:
                    predicted_outcome = self.memory[action]
                    predicted_valence = self._valences[action][predicted_outcome]
                    
                    if predicted_valence > best_valence:
                        best_valence = predicted_valence
                        best_action = action
                    elif predicted_valence == best_valence and best_action is not None:
                        import random
                        best_action = random.choice([action, best_action])
            
            if best_action is not None:
                self._action = best_action
            elif self._action is None:
                self._action = 0
        
        # TODO: Implement the agent's anticipation mechanism
        # ✅ IMPLÉMENTATION DU MÉCANISME D'ANTICIPATION
        if self._action in self.memory:
            self._predicted_outcome = self.memory[self._action]
        else:
            self._predicted_outcome = 0
        
        return self._action

## Choose the valence table

Replace the `valences` table by your choice in the code below

In [9]:
valences = [[1, -1], 
            [-1, -1],
            [-1, -1]]

## Test your agent in the TurtleEnvironment

In [10]:
initializeTurtle()

# Parameterize the rendering
bgcolor("lightGray")
penup()
goto(window_width() / 2, window_height()/2)
face(0)
pendown()
color("green")
speed(10)

a = Agent3(valences)
e = ColabTurtleEnvironment()

outcome = 0
for i in range(10):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: True
Action: 1, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1, Bored: False


## Improve your agent's code

If your agent gets stuck against a border or in a corner, modify the valences or the code. 
Try different ways to handle boredome or to select random actions. 
In the next lab, you will see how to design an agent that can adapt to the context. 

## Report 

Explain what you programmed and what results you observed. Export this document as PDF including your code, the traces you obtained, and your explanations below (no more than a few paragraphs):